In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fakerlolesportsdata/faker_2017_kespa.csv
/kaggle/input/fakerlolesportsdata/faker_2020_spring_playoffs.csv
/kaggle/input/fakerlolesportsdata/faker_2016_summer_playoffs.csv
/kaggle/input/fakerlolesportsdata/faker_champ_stats_temp.csv
/kaggle/input/fakerlolesportsdata/faker_2018_kespa_cup.csv
/kaggle/input/fakerlolesportsdata/faker_2017_rift_rivals.csv
/kaggle/input/fakerlolesportsdata/faker_2022_summer.csv
/kaggle/input/fakerlolesportsdata/faker_2017_spring_playoffs.csv
/kaggle/input/fakerlolesportsdata/faker_2017_spring.csv
/kaggle/input/fakerlolesportsdata/faker_2013_regional_finals.csv
/kaggle/input/fakerlolesportsdata/faker_2013_spring.csv
/kaggle/input/fakerlolesportsdata/faker_2020_regional_finals.csv
/kaggle/input/fakerlolesportsdata/faker_2017_summer.csv
/kaggle/input/fakerlolesportsdata/faker_2016_msi.csv
/kaggle/input/fakerlolesportsdata/faker_2018_rift_rivals.csv
/kaggle/input/fakerlolesportsdata/faker_2016_iem.csv
/kaggle/input/fakerlolesportsdata/faker_2013_sum

# Scikit-Learn

In [2]:
faker = pd.read_csv('../input/fakerlolesportsdata/faker_match_history.csv')

In [3]:
faker.head()

,Unnamed: 0,Unnamed: 0.1,Date,Tournament,Team,OpponentTeam,W/L,Blue,Red,Side,...,CSD,SpellD,SpellF,Items,Item1,Item2,Item3,Item4,Item5,Item6
0,0,0,2022-08-28,LCK 2022 Summer Playoffs,T1,Gen.G,Loss,Gen.G,T1,Red,...,-32,Teleport,Flash,"['Everfrost', 'Control Ward', ""Zhonya's Hourgl...",Everfrost,Control Ward,Zhonya's Hourglass,Banshee's Veil,Blasting Wand,Ionian Boots of Lucidity
1,1,1,2022-08-28,LCK 2022 Summer Playoffs,T1,Gen.G,Loss,T1,Gen.G,Blue,...,-46,Teleport,Flash,"['Goredrinker', 'Vampiric Scepter', 'EmptyIcon...",Goredrinker,Vampiric Scepter,EmptyIcon,Doran's Blade,Mercury's Treads,Recurve Bow
2,2,2,2022-08-28,LCK 2022 Summer Playoffs,T1,Gen.G,Loss,T1,Gen.G,Blue,...,-39,Teleport,Flash,"['Eternal Winter', ""Rabadon's Deathcap"", ""Zhon...",Eternal Winter,Rabadon's Deathcap,Zhonya's Hourglass,Void Staff,Ionian Boots of Lucidity,Dark Seal
3,3,3,2022-08-21,LCK 2022 Summer Playoffs,T1,DWG KIA,Win,T1,DWG KIA,Blue,...,-92,Teleport,Flash,"['Everfrost', 'EmptyIcon', ""Zhonya's Hourglass...",Everfrost,EmptyIcon,Zhonya's Hourglass,Sorcerer's Shoes,Banshee's Veil,Rabadon's Deathcap
4,4,4,2022-08-21,LCK 2022 Summer Playoffs,T1,DWG KIA,Loss,T1,DWG KIA,Blue,...,0,Teleport,Flash,"['Blasting Wand', 'Morellonomicon', ""Zhonya's ...",Blasting Wand,Morellonomicon,Zhonya's Hourglass,Crown of the Shattered Queen,Dark Seal,Sorcerer's Shoes


In [4]:
faker = faker.dropna()

In [5]:
X = faker.loc[:,['OpponentTeam', 'Side', 'Champion', 'Vs', 'KDA', 'Dmg', 'CSD']]
to_encode = X.loc[:,['OpponentTeam', 'Side', 'Champion', 'Vs']]
to_encode.head()

,OpponentTeam,Side,Champion,Vs
0,Gen.G,Red,Ahri,Azir
1,Gen.G,Blue,Renekton,Sylas
2,Gen.G,Blue,Ahri,Azir
3,DWG KIA,Blue,Galio,Yasuo
4,DWG KIA,Blue,Azir,Sylas


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ct = ColumnTransformer(transformers=[('encoder', ohe, [0, 1, 2, 3])], remainder='passthrough', sparse_threshold=0)
X = np.array(ct.fit_transform(X))
X

array([[  0. ,   0. ,   0. , ...,   1. ,   6.4, -32. ],
       [  0. ,   0. ,   0. , ...,   0.2,   8. , -46. ],
       [  0. ,   0. ,   0. , ...,   1.5,  13.1, -39. ],
       ...,
       [  0. ,   0. ,   0. , ...,   4. ,  12.9,  52. ],
       [  0. ,   0. ,   0. , ...,   2.5,   8.9,  42. ],
       [  0. ,   0. ,   0. , ...,   1. ,  16.8,  10. ]])

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
temp = faker['W/L']
le.fit(temp)

LabelEncoder()

In [8]:
list(le.classes_)

['Loss', 'Win']

In [9]:
y = le.transform(temp)
y

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr

LogisticRegression()

In [12]:
lr.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [13]:
lr.predict(X_test)

array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1])

In [14]:
y_test

array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1])

Note that the above is only with games where damage was recorded. If we consider games where damage was not recorded, we would have many more datapoints. That is the next step.